# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv", index_col = "City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,queenstown,-31.8976,26.8753,11.38,90,100,2.15,ZA,1702234518
1,jamestown,42.0970,-79.2353,4.77,84,100,3.09,US,1702234212
2,bethel,41.3712,-73.4140,14.07,91,100,2.57,US,1702234443
3,whitehorse,60.7161,-135.0538,-16.22,82,75,1.03,CA,1702234472
4,isafjordur,66.0755,-23.1240,-1.98,74,40,1.54,IS,1702234459


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    frame_width = 800,
    frame_height = 600,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# I know the challenge page had different filters than below, but that only gave 7 rows.
# I changed them to have more rows in the dataset.
city_data_df_narrowed = city_data_df.loc[
    (city_data_df["Max Temp"] < 30) &
    (city_data_df["Max Temp"] > 20) &
    (city_data_df["Wind Speed"] < 6) &
    (city_data_df["Cloudiness"] == 0),
    :
]

# Drop any rows with null values
city_data_df_narrowed = city_data_df_narrowed.dropna()

# Display sample data
city_data_df_narrowed

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
18,tinajo,29.0788,-13.7081,21.49,68,0,1.54,ES,1702234524
21,lompoc,34.6391,-120.4579,20.99,44,0,3.09,US,1702234525
32,adamstown,-25.0660,-130.1015,20.92,60,0,4.45,PN,1702234441
35,long beach,33.7670,-118.1892,24.10,19,0,2.06,US,1702234531
88,cabo san lucas,22.8909,-109.9124,28.20,62,0,2.06,MX,1702233822
124,lomo de arico,28.1667,-16.4833,22.28,70,0,2.06,ES,1702234565
130,mukalla,14.5425,49.1242,23.76,68,0,0.72,YE,1702234395
161,urdinarrain,-32.6857,-58.8932,29.26,29,0,3.71,AR,1702234579
211,necochea,-38.5473,-58.7368,23.80,43,0,5.63,AR,1702234398


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_narrowed.loc[
    :,
    ["City", "Country", "Lat", "Lng", "Humidity"]
]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
18,tinajo,ES,29.0788,-13.7081,68,
21,lompoc,US,34.6391,-120.4579,44,
32,adamstown,PN,-25.0660,-130.1015,60,
35,long beach,US,33.7670,-118.1892,19,
88,cabo san lucas,MX,22.8909,-109.9124,62,
124,lomo de arico,ES,28.1667,-16.4833,70,
130,mukalla,YE,14.5425,49.1242,68,
161,urdinarrain,AR,-32.6857,-58.8932,29,
211,necochea,AR,-38.5473,-58.7368,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20,
    "lang": "en"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tinajo - nearest hotel: Villas Tinajo
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
adamstown - nearest hotel: No hotel found
long beach - nearest hotel: Westin
cabo san lucas - nearest hotel: Comfort Rooms
lomo de arico - nearest hotel: No hotel found
mukalla - nearest hotel: فندق الجابري
urdinarrain - nearest hotel: Palmera Hotel
necochea - nearest hotel: Cabañas Río Quequén
bitkine - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
berbera - nearest hotel: Al Madina Hotel
omdurman - nearest hotel: فندق مزدلفة السياحي
los angeles - nearest hotel: DoubleTree by Hilton Hotel Los Angeles Downtown
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
18,tinajo,ES,29.0788,-13.7081,68,Villas Tinajo
21,lompoc,US,34.6391,-120.4579,44,Embassy Suites by Hilton Lompoc Central Coast
32,adamstown,PN,-25.0660,-130.1015,60,No hotel found
35,long beach,US,33.7670,-118.1892,19,Westin
88,cabo san lucas,MX,22.8909,-109.9124,62,Comfort Rooms
124,lomo de arico,ES,28.1667,-16.4833,70,No hotel found
130,mukalla,YE,14.5425,49.1242,68,فندق الجابري
161,urdinarrain,AR,-32.6857,-58.8932,29,Palmera Hotel
211,necochea,AR,-38.5473,-58.7368,43,Cabañas Río Quequén


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)